In [156]:
import numpy as np
import pandas as pd
from pycirclize import Circos

In [345]:
df = pd.read_excel('../data/processed_data.xlsx', header=0, index_col=0)

In [346]:
df.head()

,GPm,GPl,SI,MA,MS,NDB,TRS,BSTam,BSTfu,BSTv,...,NTSco.1,AP.1,GR.1,Z.1,CU.1,ACU.1,PAT.1,SPVO.1,SPVI.1,SPVC.1
Abbr.,,,,,,,,,,,,,,,,,,,,,
GPm,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GPl,6,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SI,0,2,0,3,0,2,0,3,0,3,...,0,0,0,0,0,0,0,0,0,0
MA,0,0,1,0,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MS,0,0,2,2,0,5,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [391]:
hippocampal_connections = ['CA3','CA2','CA1v','CA1d','SUBv','SUBd','PRE','POST','PAR', 'DG'] 

In [348]:
brain_regions = df.index.to_numpy()

## Left hemisphere hippocampal connectivity plots

In [380]:
# each region is duplicated across right and left hemisphere,
# so let's grab the unique regions
left_hemisphere = brain_regions[:392]

In [381]:
df_left_hemisphere = df[left_hemisphere]
df_left_hemisphere = df_left_hemisphere.head(392)

In [382]:
df_left_hemisphere

,GPm,GPl,SI,MA,MS,NDB,TRS,BSTam,BSTfu,BSTv,...,NTSco,AP,GR,Z,CU,ACU,PAT,SPVO,SPVI,SPVC
Abbr.,,,,,,,,,,,,,,,,,,,,,
GPm,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GPl,6,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SI,0,2,0,3,0,2,0,3,0,3,...,0,0,0,0,0,0,0,0,0,0
MA,0,0,1,0,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MS,0,0,2,2,0,5,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PAT,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,3,3,0
SPVO,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,3,0,0,0,0,0


In [383]:
# construct 9x9 matrix of just the hippocampal regions from left hemisphere matrix
df_lh_rc_hippo = df_left_hemisphere[df_left_hemisphere.index.isin(hippocampal_connections)] #rows
df_lh_rc_hippo = df_lh_rc_hippo.T[df_lh_rc_hippo.T.index.isin(hippocampal_connections)] #cols

In [390]:
# plot these regions for hippocampus in left hemisphere
circos_lh_hc = Circos.initialize_from_matrix(df_lh_rc_hippo, cmap="jet")
circos_lh_hc.savefig("hippocampus_left_hemisphere_connectivity.svg")

## Inter-hemispheric hippocampal connectivity plots

Rather than just look at the left hemisphere, we can look at both hemispheres. There are 10 regions of interest within the hippocampus, but the duplication results in 20 rows of data. We will combine these regions 
1. First by averaging across values
2. By taking maximum value between them

In [398]:
# duplicate columns are ingested in pandas with '.1' appended to column name
df.columns = df.columns.str.replace(r'\.1$', '', regex=True)

# we are going to sum across hemispheres in rows - REDACTED
df_unique_r = df.groupby(df.index).sum()
df_unique_rc = df_unique_r.T.groupby(df_unique_r.T.index).sum()

In [399]:
# grab just the hippocampal regions from the reduced summed matrix
df_unique_rc_hippo = df_unique_rc[df_unique_rc.index.isin(hippocampal_connections)] #rows
df_unique_rc_hippo = df_unique_rc_hippo.T[df_unique_rc_hippo.T.index.isin(hippocampal_connections)] #cols

In [400]:
# this is the larger 18x18 matrix that includes all hippocampal regions across hemispheres

# filter dataframe so that rows only include hippocampal
df_r_hc = df[df.index.isin(hippocampal_connections)]
df_rc_hc = df_r_hc.loc[:, df_r_hc.columns.isin(hippocampal_connections)]

In [401]:
df_average_hc = df_rc_hc.groupby(df_rc_hc.index).mean()
df_average_hc = df_average_hc.T.groupby(df_average_hc.T.index).mean().T

df_max_hc = df_rc_hc.groupby(df_rc_hc.index).max()
df_max_hc = df_max_hc.T.groupby(df_max_hc.T.index).max().T

In [341]:
# create circular visualization of these hippocampal matricies
circos_max_hc = Circos.initialize_from_matrix(df_max_hc)
circos_average_hc = Circos.initialize_from_matrix(df_average_hc)

circos_max_hc.savefig("hippocampus_max_connectivity.svg")
circos_average_hc.savefig("hippocampus_average_connectivity.svg")

## Generate connection rankings for left hemisphere

We'll conglomerate all hippocampal subregions into one thing by taking the maximum value across the 10 rows. Then we generate two lists of ranking:

1. What is the hippocampus most strongly projecting to? This is called the to-connectivity
2. What regions are most strongly projecting to the hippocampus? This is called the from-connectivity

In [ ]:
# average across identical regions for within hippocampal connections
# then create plot of 9x9 matrix and visualize

# create the same thing but instead of averaging do the max

# treat all hippocampal subregions as one thing... so average all values across the 9 rows
# then two lists (ranking strengths):
# 1. to-connectivity (average across columns)
# 2. from-connectivity (average across rows)

#### First create from-connectivity dataframe
We use the reduced summed matrix called df_unique_rc (all connections are summed across hemispheres, so it turns the 784x784 matrix into 392x392 matrix)

In [405]:
df_from_connectivity = df_left_hemisphere[df_left_hemisphere.index.isin(hippocampal_connections)]

In [407]:
hippocampus = df_from_connectivity.max(axis=0)
df_from_connectivity.loc["from_hippocampus"] = hippocampus

/var/folders/fl/v0by8jd51091hqvkhj8fr20m0000gn/T/ipykernel_6322/3708292641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_from_connectivity.loc["from_hippocampus"] = hippocampus


In [409]:
# drop extra rows
df_from_connectivity = df_from_connectivity.loc[~df_from_connectivity.index.isin(hippocampal_connections)]

In [410]:
df_from_connectivity

,GPm,GPl,SI,MA,MS,NDB,TRS,BSTam,BSTfu,BSTv,...,NTSco,AP,GR,Z,CU,ACU,PAT,SPVO,SPVI,SPVC
Abbr.,,,,,,,,,,,,,,,,,,,,,
from_hippocampus,0,0,2,0,3,3,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0


In [411]:
# take the transpose so you can sort rows and sort based on values
df_from_connectivity_t = df_from_connectivity.T

In [412]:
df_from_connectivity_sorted = df_from_connectivity_t.sort_values(by='from_hippocampus', ascending=False)

In [413]:
df_from_connectivity_sorted.head()

Abbr.,from_hippocampus
LSv,7
LHAjp,7
ENTm,7
ECT,7
SUBd,7


#### Next create to-connectivity dataframe

In [414]:
# make sure only hippocampus in the columns
df_c_hc = df_left_hemisphere.loc[:, df_left_hemisphere.columns.isin(hippocampal_connections)]

In [417]:
# take the transpose of the dataframe and add the averaged columns
hippocampus_to = df_c_hc.max(axis=1)
df_c_hc_t = df_c_hc.T
df_c_hc_t.loc["to_hippocampus"] = hippocampus_to

In [419]:
# remove rows (they were columns) that represent individual parts of the hippocampus
df_to_connectivity = df_c_hc_t.loc[~df_c_hc_t.index.isin(hippocampal_connections)]

In [421]:
# take the transpose here and sort columns
df_to_connectivity = df_to_connectivity.T.sort_values(by='to_hippocampus', ascending=False)

In [422]:
df_to_connectivity.head()

,to_hippocampus
Abbr.,
CA1v,7
LA,7
COApm,7
CA1d,7
ENTm,7


In [176]:
# re-reading the paper and it appears the repeat brain regions appear because it's
# divided into left and right sides of the brain

### Clean up: Merging with naming data and Saving
We want the rank dataframes to have index abbreviations and also the region name included for ease of reading

In [423]:
# map index to names and general region

names_df = pd.read_excel('../data/naming_doc.xlsx', header=2)
names_df = names_df.rename(columns={"Name.1":"Region", "Abbreviation.1":"Abbrev"})

In [424]:
names_df

,Number,Name,Abbreviation,Abbrev,Region,(blank if no difference)
0,1,Cerebral nuclei,CNU,GPm,Medial globus pallidus,NaN
1,2,Cerebral nuclei,CNU,GPl,Lateral globus pallidus,NaN
2,3,Cerebral nuclei,CNU,SI,Innominate substance,NaN
3,4,Cerebral nuclei,CNU,MA,Magnocellular nucleus,NaN
4,5,Cerebral nuclei,CNU,MS,Medial septal nucleus,NaN
...,...,...,...,...,...,...
387,391,Afterbrain or Medulla,AFB or MY,ACU,Accessory cuneate nucleus,NaN
388,392,Afterbrain or Medulla,AFB or MY,PAT,Paratrigeminal nucleus,NaN
389,393,Afterbrain or Medulla,AFB or MY,SPVO,"Spinal nucleus of trigeminal nerve, Oral part",NaN
390,394,Afterbrain or Medulla,AFB or MY,SPVI,"Spinal nucleus of trigeminal nerve, Interpolar...",NaN


In [429]:
# from_connections
df_from_connectivity_with_index = df_from_connectivity_sorted.reset_index().rename(columns={"index":"Abbrev"})

df_rank_from_hippocampus_connections = pd.merge(
    df_from_connectivity_with_index, names_df, on='Abbrev', how='inner')

df_rank_from_hippocampus_connections = df_rank_from_hippocampus_connections.drop(
    ['Number', 'Abbreviation', '(blank if no difference)'], axis=1)

In [430]:
df_rank_from_hippocampus_connections

,Abbrev,from_hippocampus,Name,Region
0,LSv,7,Cerebral nuclei,"Lateral septal nucleus, Ventral part"
1,LHAjp,7,Hypothalamus,"Lateral hypothalamic area, Juxtaparaventricula..."
2,ENTm,7,Cerebral cortex,"Entorhinal area, Medial part"
3,ECT,7,Cerebral cortex,Ectorhinal area
4,SUBd,7,Cerebral cortex,"Subiculum, Dorsal part"
...,...,...,...,...
387,DR,0,Midbrain,Dorsal raphe nucleus
388,SNc,0,Midbrain,"Substantia nigra, Compact part"
389,SNr,0,Midbrain,"Substantia nigra, Reticular part"
390,RN,0,Midbrain,Red nucleus


In [432]:
df_to_connectivity.reset_index()

,Abbr.,to_hippocampus
0,CA1v,7
1,LA,7
2,COApm,7
3,CA1d,7
4,ENTm,7
...,...,...
387,ZI,0
388,FF,0
389,LGv,0
390,IGL,0


In [433]:
# to_connections
df_to_connectivity_with_index = df_to_connectivity.reset_index().rename(columns={"Abbr.":"Abbrev"})

df_rank_to_hippocampus_connections = pd.merge(df_to_connectivity_with_index, names_df, on='Abbrev', how='inner')

df_rank_to_hippocampus_connections = df_rank_to_hippocampus_connections.drop(
    ['Number', 'Abbreviation', '(blank if no difference)'], axis=1)

In [434]:
df_rank_to_hippocampus_connections

,Abbrev,to_hippocampus,Name,Region
0,CA1v,7,Cerebral cortex,"Field CA1, Ventral part"
1,LA,7,Cerebral cortex,Lateral amygdalar nucleus
2,COApm,7,Cerebral cortex,"Cortical amygdala area, Posterior part, Medial..."
3,CA1d,7,Cerebral cortex,"Field CA1, Dorsal part"
4,ENTm,7,Cerebral cortex,"Entorhinal area, Medial part"
...,...,...,...,...
387,ZI,0,Thalamus,Zona incerta
388,FF,0,Thalamus,Fields of Forel
389,LGv,0,Thalamus,Ventral lateral geniculate nucleus
390,IGL,0,Thalamus,Intergeniculate leaflet


In [435]:
# save both dataframes to excel files
df_rank_from_hippocampus_connections.to_excel('../data/rank_from_hippocampus_connections_max.xlsx')
df_rank_to_hippocampus_connections.to_excel('../data/rank_to_hippocampus_connections_max.xlsx')

## Hippocampal - Hypothalamus connectivity plots

Next we will look at four regions (AHA, LHA, DHA, and mammillary) and their connectivity with the hippocampus. Each of the four regions is an aggregation across other subregions, so we start by summing across the subregions.

In [576]:
# construct regions
AHA_regions = np.array(['AHA', 'AHNa', 'AHNc', 'AHNp', 'AHNd'])
DHA_regions = np.array(['DMHa','DMHp','DMHv'])
mammillary_regions = np.array(['TM','MM','LM','SUMm','SUMl'])
LHA_regions = np.array([
    'LHAa','LHAjvv','LHAjvd','LHAjp','LHAjd','LHAsfa','LHAsfp','LHAs','LHAv','LHAd','LHAp'
])

hippocampus_regions = np.array(['DG','CA3','CA2','CA1v','CA1d'])

hypothalamus_regions = np.concatenate(
    (AHA_regions, DHA_regions, mammillary_regions, LHA_regions), axis=0
)

hyp_hipp_regions = np.concatenate((hypothalamus_regions, hippocampus_regions), axis=0)

In [577]:
# reduce the left hemisphere dataframe to only include hypothalamus and hippocampal regions
df_hyp_hipp = df_left_hemisphere[hyp_hipp_regions]
df_hyp_hipp = df_hyp_hipp[df_hyp_hipp.index.isin(hyp_hipp_regions)]

In [578]:
#LHA
# rows
LHA = df_hyp_hipp[df_hyp_hipp.index.isin(LHA_regions)].sum(axis=0)
df_hyp_hipp.loc["LHA"] = LHA

# delete extra LHA subregions from rows
df_hyp_hipp = df_hyp_hipp.loc[
~df_hyp_hipp.index.isin(LHA_regions)
]

In [579]:
# columns
LHA_col_sum = df_hyp_hipp[LHA_regions].sum(axis=1)
df_hyp_hipp["LHA"] = LHA_col_sum

# delete extra LHA subregion from columns
df_hyp_hipp = df_hyp_hipp.drop(LHA_regions, axis=1)

In [580]:
# DHA
DHA_row_sum = df_hyp_hipp[df_hyp_hipp.index.isin(DHA_regions)].sum(axis=0)
df_hyp_hipp.loc["DHA"] = DHA_row_sum

# delete extra DHA subregions from rows
df_hyp_hipp = df_hyp_hipp.loc[
~df_hyp_hipp.index.isin(DHA_regions)
]

# columns
DHA_col_sum = df_hyp_hipp[DHA_regions].sum(axis=1)
df_hyp_hipp.loc[:,"DHA"] = DHA_col_sum

# delete extra LHA subregion from columns
df_hyp_hipp = df_hyp_hipp.drop(DHA_regions, axis=1)

In [581]:
# mammillary
mammillary_row_sum = df_hyp_hipp[df_hyp_hipp.index.isin(mammillary_regions)].sum(axis=0)
df_hyp_hipp.loc["mammillary"] = mammillary_row_sum

# delete extra DHA subregions from rows
df_hyp_hipp = df_hyp_hipp.loc[
~df_hyp_hipp.index.isin(mammillary_regions)
]

# columns
mammillary_col_sum = df_hyp_hipp[mammillary_regions].sum(axis=1)
df_hyp_hipp.loc[:,"mammillary"] = mammillary_col_sum

# delete extra LHA subregion from columns
df_hyp_hipp = df_hyp_hipp.drop(mammillary_regions, axis=1)

In [582]:
# AHA
AHA_row_sum = df_hyp_hipp[df_hyp_hipp.index.isin(AHA_regions)].sum(axis=0)
df_hyp_hipp.loc["AHA_1"] = AHA_row_sum

# delete extra DHA subregions from rows
df_hyp_hipp = df_hyp_hipp.loc[
~df_hyp_hipp.index.isin(AHA_regions)
]

# columns
AHA_col_sum = df_hyp_hipp[AHA_regions].sum(axis=1)
df_hyp_hipp.loc[:,"AHA_1"] = AHA_col_sum

# delete extra LHA subregion from columns
df_hyp_hipp = df_hyp_hipp.drop(AHA_regions, axis=1)

In [583]:
# rename 'AHA_1' to 'AHA'
df_hyp_hipp = df_hyp_hipp.rename(index={"AHA_1":"AHA"})
df_hyp_hipp = df_hyp_hipp.rename(columns={"AHA_1":"AHA"})

In [584]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,
DG,0,6,0,0,0,0,0,0,0
CA3,3,0,6,6,6,2,0,0,0
CA2,4,4,0,4,0,0,0,0,0
CA1v,2,1,2,0,0,27,1,4,7
CA1d,2,0,0,0,0,11,0,0,0
LHA,0,3,2,6,0,284,72,67,110
DHA,3,3,0,3,0,80,23,33,27
mammillary,9,6,6,8,3,64,30,33,20
AHA,0,2,1,2,0,99,25,19,27


In [585]:
for i in range(len(df_hyp_hipp.index)):
    df_hyp_hipp.iloc[i,i] = 0

In [586]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,
DG,0,6,0,0,0,0,0,0,0
CA3,3,0,6,6,6,2,0,0,0
CA2,4,4,0,4,0,0,0,0,0
CA1v,2,1,2,0,0,27,1,4,7
CA1d,2,0,0,0,0,11,0,0,0
LHA,0,3,2,6,0,0,72,67,110
DHA,3,3,0,3,0,80,0,33,27
mammillary,9,6,6,8,3,64,30,0,20
AHA,0,2,1,2,0,99,25,19,0


In [587]:
circos_hyp_hipp = Circos.initialize_from_matrix(df_hyp_hipp, cmap="tab10")

circos_hyp_hipp.savefig("hippocampus_hypothalamus_connectivity.svg")

In [588]:
# we need to remove connections within the hypothalamus--
# we're only interested in hippocampal-hypothalamus connectivity

hypothalamus_aggregated_regions = np.array(["LHA",'DHA', 'mammillary', 'AHA'])
for subregion_i in range(len(hypothalamus_aggregated_regions)):
    for subregion_j in range(len(hypothalamus_aggregated_regions)):
        df_hyp_hipp.loc[
        hypothalamus_aggregated_regions[subregion_i], 
        hypothalamus_aggregated_regions[subregion_j]
        ] = 0

In [589]:
df_hyp_hipp.loc["AHA","DHA"]

np.int64(0)

In [592]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,
DG,0,6,0,0,0,0,0,0,0
CA3,3,0,6,6,6,2,0,0,0
CA2,4,4,0,4,0,0,0,0,0
CA1v,2,1,2,0,0,27,1,4,7
CA1d,2,0,0,0,0,11,0,0,0
LHA,0,3,2,6,0,0,0,0,0
DHA,3,3,0,3,0,0,0,0,0
mammillary,9,6,6,8,3,0,0,0,0
AHA,0,2,1,2,0,0,0,0,0


In [593]:
circos_hyp_hipp_only = Circos.initialize_from_matrix(df_hyp_hipp, cmap="tab10")

circos_hyp_hipp_only.savefig("hippocampus_hypothalamus_connectivity_only.svg")

In [594]:
#clear out intrinsic connections between hippocampus_regions
for subregion_i in range(len(hippocampus_regions)):
    for subregion_j in range(len(hippocampus_regions)):
        df_hyp_hipp.loc[
        hippocampus_regions[subregion_i], 
        hippocampus_regions[subregion_j]
        ] = 0

In [595]:
df_hyp_hipp

,DG,CA3,CA2,CA1v,CA1d,LHA,DHA,mammillary,AHA
Abbr.,,,,,,,,,
DG,0,0,0,0,0,0,0,0,0
CA3,0,0,0,0,0,2,0,0,0
CA2,0,0,0,0,0,0,0,0,0
CA1v,0,0,0,0,0,27,1,4,7
CA1d,0,0,0,0,0,11,0,0,0
LHA,0,3,2,6,0,0,0,0,0
DHA,3,3,0,3,0,0,0,0,0
mammillary,9,6,6,8,3,0,0,0,0
AHA,0,2,1,2,0,0,0,0,0


In [598]:
circos_hyp_hipp_extrinsic_only = Circos.initialize_from_matrix(df_hyp_hipp, cmap="tab10")

circos_hyp_hipp_extrinsic_only.savefig("hippocampus_hypothalamus_extrinsic_connectivity.svg")